In [1]:
from utils import *

Setup_environment()

from hpe_wrapper import Wrapper_2Dpose, Wrapper_3Dpose

model_2D ='./detectron/configs/COCO-Keypoints/keypoint_rcnn_R_101_FPN_3x.yaml'
weights_2D = './detectron/checkpoint/model_final_997cc7.pkl'
model_3D = './VideoPose3D/checkpoint/pretrained_h36m_detectron_coco.bin'

pose2d = Wrapper_2Dpose(model=model_2D, weights= weights_2D , ROI_thr=0.75)
pose_3d = Wrapper_3Dpose(model_3D)

config  = load_config()
print(config['video_object'])

./data/videos/remastered_omco_pickingup_1 (video-converter.com).mp4


In [2]:
video_object = Video_wrapper(config['video_object'], resize_video_by=0.3) #, start=0, end=10) # downsample_fps_by=2,
data_2d, metadata_vid = pose2d.predict_2D_poses(input_video_object=video_object)
data_3d = pose_3d.predict_3D_poses(data_2d, metadata_vid)

Frame 0 processed in 1.043s
Frame 1 processed in 0.078s
Frame 2 processed in 0.091s
Frame 3 processed in 0.094s
Frame 4 processed in 0.102s
Frame 5 processed in 0.106s
Frame 6 processed in 0.090s
Frame 7 processed in 0.083s
Frame 8 processed in 0.083s
Frame 9 processed in 0.092s
Frame 10 processed in 0.092s
Frame 11 processed in 0.085s
Frame 12 processed in 0.081s
Frame 13 processed in 0.078s
Frame 14 processed in 0.085s
Frame 15 processed in 0.086s
Frame 16 processed in 0.090s
Frame 17 processed in 0.093s
Frame 18 processed in 0.092s
Frame 19 processed in 0.077s
Frame 20 processed in 0.091s
Frame 21 processed in 0.092s
Frame 22 processed in 0.078s
Frame 23 processed in 0.085s
Frame 24 processed in 0.090s
Frame 25 processed in 0.077s
Frame 26 processed in 0.083s
Frame 27 processed in 0.082s
Frame 28 processed in 0.079s
Frame 29 processed in 0.082s
Frame 30 processed in 0.086s
Frame 31 processed in 0.082s
Frame 32 processed in 0.086s
Frame 33 processed in 0.093s
Frame 34 processed in 0.

In [3]:
from tensor_ergonomics import RULAXXX
original_pose = torch.tensor(data_3d)

pose = original_pose.clone().detach().requires_grad_(True)
print(pose.requires_grad)

rula_eval = RULAXXX(pose)

scores = rula_eval.compute_scores()
print(scores.requires_grad)

#rula_eval.plot_scores(fps=video_object.fps)

True
--- Frame 0: ---
Frame: tensor([ 14.3200, 111.9615,  36.7538, 119.7843,  29.0617,  43.0275,   9.8064,
         23.7113,  14.8478,   1.2035,  91.7161,  35.1584,  99.9522,  98.7746])
Max shoulder: tensor(36.7538)
Max abduction: tensor(119.7843)
Max elbow: tensor(43.0275)
Score Upper Arm: tensor([2.])
Score Lower Arm: tensor([1.])
Current Score A: tensor(2.)
Score Neck: tensor([1.])
Score Trunk: tensor([1.])
Min knee: tensor(9.8064)
Score Legs: tensor([1.])


UnboundLocalError: local variable 'curr_score_B' referenced before assignment

In [4]:
scores[0]

NameError: name 'scores' is not defined

## Pose Optim  Loop

In [5]:
import torch
from xxx_ergonomics import RULAXXX
not this one
def run_pose_optimization(original_pose, num_steps=300):
    """
    Optimize the pose using the RULA score.
    
    Parameters:
    - original_pose: A tensor of shape (17, 3)
    
    Returns:
    - Optimized pose.
    """

    # Ensure the original_pose is a tensor
    if not isinstance(original_pose, torch.Tensor):
        original_pose = torch.tensor(original_pose)

    pose = original_pose.clone().detach().requires_grad_(True)

    # Define an optimizer. Here, we use the Adam optimizer.
    optimizer = torch.optim.Adam([pose], lr=0.01) #(1,200t, 17, 3)

    for step in range(num_steps):
        optimizer.zero_grad()

        # Convert the tensor to the format expected by RULA (ndarray)
        #pose_for_rula = pose.detach().cpu().numpy()

        # Compute the RULA score for the current pose            
        rula_eval = RULAXXX(pose)
        rula_score = rula_eval.compute_scores()
        print(rula_score.requires_grad)
        ergo_sum = rula_score.sum()
        print(ergo_sum.requires_grad)

        # Convert the RULA score back to a tensor
        #rula_score_tensor = torch.tensor(ergo_sum, dtype=torch.float32)

        # We want to minimize the RULA score, so we negate it for optimization
        loss = ergo_sum

        # Backpropagate the loss
        loss.backward()

        # Update the pose
        optimizer.step()

        # Print the RULA score every 50 steps
        if step % 50 == 0:
            print(f"Step {step}: RULA Score: {ergo_sum.item()}")

    return pose



optimized_pose = run_pose_optimization(data_3d)


SyntaxError: invalid syntax (3731129540.py, line 3)

In [ ]:
from ergonomics_torch import RULATorch
aaa
# Ensure the original_pose is a tensor
if not isinstance(data_3d, torch.Tensor):
    original_pose = torch.tensor(data_3d)

pose = original_pose.clone().detach().requires_grad_(True)

rula_eval = RULATorch(original_pose)
print(type(original_pose))
scores = rula_eval.compute_scores()

rula_eval.plot_scores(fps=video_object.fps)

In [ ]:
import torch
from torch.autograd import Function
from ergonomics import RULA

class RULALossFunction(Function):
    @staticmethod
    def forward(ctx, pose):
        pose_np = pose.detach().cpu().numpy()
        rula_eval = RULA(pose_np)
        rula_score = rula_eval.compute_scores()
        ctx.save_for_backward(pose)
        return torch.tensor(rula_score, dtype=torch.float32)

    @staticmethod
    def backward(ctx, grad_output):
        # This is a dummy backward function. It assumes that the gradient
        # with respect to the pose is not dependent on the RULA score.
        # You might need to adjust this based on your actual requirements.
        pose, = ctx.saved_tensors
        grad_pose = torch.zeros_like(pose)
        return grad_pose

def run_pose_optimization(original_pose, num_steps=300):
    """
    Optimize the pose using the RULA score.
    
    Parameters:
    - original_pose: A tensor of shape (17, 3)
    
    Returns:
    - Optimized pose.
    """

    # Ensure the original_pose is a tensor
    if not isinstance(original_pose, torch.Tensor):
        original_pose = torch.tensor(original_pose)

    pose = original_pose.clone().detach().requires_grad_(True)

    # Define an optimizer. Here, we use the Adam optimizer.
    optimizer = torch.optim.Adam([pose], lr=0.01)

    for step in range(num_steps):
        optimizer.zero_grad()

        # Compute the RULA score for the current pose using the custom function
        rula_score_tensor = RULALossFunction.apply(pose)

        # We want to minimize the RULA score, so we negate it for optimization
        loss = -rula_score_tensor

        # Backpropagate the loss
        loss.backward()

        # Update the pose
        optimizer.step()

        # Print the RULA score every 50 steps
        if step % 50 == 0:
            print(f"Step {step}: RULA Score: {rula_score_tensor.item()}")

    return pose

original_pose = data_dict['kps'][:1,::][0]
original_pose_tensor = torch.tensor(original_pose, dtype=torch.float32, requires_grad=True)
print(original_pose_tensor.shape) #torch.Size([17, 3])
optimized_pose = run_pose_optimization(original_pose_tensor)


In [ ]:
import torch
from ergonomics import RULA

def run_pose_optimization(original_pose, num_steps=300):
    """
    Optimize the pose using the RULA score.
    
    Parameters:
    - original_pose: A tensor of shape (17, 3)
    
    Returns:
    - Optimized pose.
    """

    # Initialize the pose to be optimized
    pose = original_pose.clone().detach().requires_grad_(True)

    # Define an optimizer. Here, we use the Adam optimizer.
    optimizer = torch.optim.Adam([pose], lr=0.01)

    for step in range(num_steps):
        optimizer.zero_grad()

        # Compute the RULA score for the current pose            
        rula_eval = RULA(original_pose)
        print(rula_eval)
        rula_score = rula_eval.compute_scores()
        
        # We want to minimize the RULA score, so we negate it for optimization
        loss = -rula_score

        # Backpropagate the loss
        loss.backward()

        # Update the pose
        optimizer.step()

        # Print the RULA score every 50 steps
        if step % 50 == 0:
            print(f"Step {step}: RULA Score: {rula_score.item()}")

    return pose


# Optimize the pose
original_pose = data_dict['kps'][:1,::][0]
original_pose_tensor = torch.tensor(original_pose, dtype=torch.float32, requires_grad=True)
print(original_pose_tensor.shape) #torch.Size([17, 3])
optimized_pose = run_pose_optimization(original_pose_tensor)
